In [7]:
!pip install openai-whisper
!pip install pandas
!pip install librosa
!pip install soundfile
!pip install datasets




[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Owner\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Owner\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Owner\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Owner\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/e2/cf/db41e572d7ed958e8679018f8190438ef700aeb501b62da9e1eed9e4d69a/datasets-2.15.0-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow>=8.0.0 from https://files.pythonhosted.org/packages/d4/f0/607f50ec87ac4775d6124855ae6be2c48bab58aa0a660ccd46e9af52bcd9/pyarrow-14.0.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for pyarrow-hotfix from https://files.pythonhosted.org/packages/e4/f4/9ec2222f5f5f8ea04f66f184caafd991a39c8782e31f5b0266f101cb68ca/pyarrow_hotfix-0.6-py3-none-any.whl.metadata
  Obtaining dependency information for dill<0.3.8,>=0.3.0 from https://files.pythonhosted.org/packages/f5/3a/74a29b11cf2cdfcd6ba89c0cecd70b37cd1ba7b77978ce611eb7a146a832/dill-0.3.7-py3-none-any.whl.metadata
  Obtaining dependency information for xxhash from https://files.pythonhosted.org/packages/b7/3a/74a609706ef4430fe6d041a3b8d209882c15440b695e373fe26d48c6f35c/xxhash


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Owner\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [8]:
import whisper
import pandas as pd
import librosa
import soundfile as sf
import datasets

In [10]:
import whisper
import datasets

# Load the dataset
dataset = datasets.load_dataset("CAiRE/ASCEND")

# Initialize Whisper model
model = whisper.load_model("base")  # Choose the appropriate model size

def get_language_probability(audio_path):
    """
    This function uses Whisper to predict the language and its probability
    for a given audio clip.
    """
    result = model.transcribe(audio_path)
    languages = result["language"]
    # Assuming 'zh' is the code for Chinese
    chinese_probability = languages.get('zh', 0)
    return chinese_probability

# Accessing the first audio clip from the 'train' split
first_audio = dataset['train'][0]['audio']['path']
probability = get_language_probability(first_audio)

print(f"Probability of Chinese: {probability}")

# For binary search implementation:
# Sort the dataset based on Chinese language probability and then apply binary search.



C:\Users\Owner\AppData\Local\Programs\Python\Python311\Lib\site-packages\whisper\transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [ ]:
model = whisper.load_model("base")
audio = whisper.load_audio("en+es.m4a")
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).to(model.device)

In [ ]:
_, probs = model.detect_language(mel)

# Sorting the dictionary items by values in descending order
sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)

# Creating a new list with the sorted key-value pairs
sorted_probs = [{'key': key, 'value': value} for key, value in sorted_probs]

print(sorted_probs)